In [1]:
import os
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
from scipy.misc import imresize
from scipy import ndimage

%matplotlib inline

import random
import csv
import re
import pickle
import json
from tqdm import tqdm

DATADIR = os.path.expanduser('~/data/kaggle/ultrasound-nerve-segmentation')
PROCDIR = os.path.expanduser('~/ml/kaggle/ultrasound-nerve-segmentation/processed')
MODELDIR = os.path.expanduser('~/ml/kaggle/ultrasound-nerve-segmentation/models')

In [2]:
with open(os.path.join(PROCDIR, 'train_files_map.pkl'), 'rb') as f:
    train_files_map = pickle.load(f)
with open(os.path.join(PROCDIR, 'test_files_map.pkl'), 'rb') as f:
    test_files_map = pickle.load(f)

## create dataset

In [3]:
orig_img_rows, orig_img_cols = 420, 580
img_rows, img_cols = 128, 192
img_channels = 1

# 90/10 train/val split, by subject
data_images_train = []
data_masks_train = []
data_images_val = []
data_masks_val = []

subjects = set([f['subject'] for f in train_files_map.values()])
subjects_val = np.random.choice(list(subjects), int(len(subjects) * 0.1), replace=False)

for file_info in tqdm(train_files_map.values()):
    subject, img, folder, img_file, mask_file = (file_info['subject'], 
                                                 file_info['img'], 
                                                 file_info['folder'], 
                                                 file_info['img_file'], 
                                                 file_info['mask_file'])
    img_filepath = os.path.join(folder, img_file)
    mask_filepath = os.path.join(folder, mask_file)
    image = io.imread(img_filepath) / 255.0
    mask = io.imread(mask_filepath) / 255.0
    image_resized = imresize(image, size=(img_rows, img_cols), interp='bilinear')
    mask_resized = imresize(mask, size=(img_rows, img_cols), interp='bilinear').astype(np.bool)
    if subject in subjects_val:
        data_images_val.append(image_resized)
        data_masks_val.append(mask_resized)
    else:
        data_images_train.append(image_resized)
        data_masks_train.append(mask_resized)

data_images_train = np.expand_dims(np.array(data_images_train, dtype=np.float32), axis=3)
data_masks_train = np.array(data_masks_train, dtype=np.bool)
data_images_val = np.expand_dims(np.array(data_images_val, dtype=np.float32), axis=3)
data_masks_val = np.array(data_masks_val, dtype=np.bool)

print('image shapes:', data_images_train.shape, data_images_val.shape)
print('mask shapes:', data_masks_train.shape, data_masks_val.shape)

  1%|▏         | 73/5635 [00:01<01:59, 46.69it/s]/home/leon/install/miniconda3/lib/python3.5/site-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
100%|██████████| 5635/5635 [02:03<00:00, 45.72it/s]


image shapes: (5155, 128, 192, 1) (480, 128, 192, 1)
mask shapes: (5155, 128, 192) (480, 128, 192)


## augmentation

In [7]:
multiplier = 4

data_images_train_augment = []
data_masks_train_augment = []
for i in tqdm(range(data_images_train.shape[0])):
    data_images_train_augment.append(data_images_train[i])
    data_masks_train_augment.append(data_masks_train[i])
    for j in range(multiplier):
        y_shift = random.randint(int(data_images_train.shape[1] * -0.125), 
                                 int(data_images_train.shape[1] * 0.125))
        x_shift = random.randint(int(data_images_train.shape[2] * -0.125), 
                                 int(data_images_train.shape[2] * 0.125))
        image_aug = ndimage.interpolation.shift(data_images_train[i], (y_shift, x_shift, 0), order=0)
        mask_aug = ndimage.interpolation.shift(data_masks_train[i], (y_shift, x_shift), order=0)
        data_images_train_augment.append(image_aug)
        data_masks_train_augment.append(mask_aug)

data_images_train = np.array(data_images_train_augment, dtype=np.float32)
data_masks_train = np.array(data_masks_train_augment, dtype=np.bool)

print('image shapes:', data_images_train.shape, data_images_val.shape)
print('mask shapes:', data_masks_train.shape, data_masks_val.shape)

100%|██████████| 5155/5155 [00:12<00:00, 423.35it/s]


image shapes: (25775, 128, 192, 1) (480, 128, 192, 1)
mask shapes: (25775, 128, 192) (480, 128, 192)


## save

In [4]:
with open(os.path.join(PROCDIR, 'data_train_val_01.pkl'), 'wb') as f:
    pickle.dump((data_images_train, data_masks_train, data_images_val, data_masks_val), f, protocol=4)